<a href="https://colab.research.google.com/github/rcazevedo/Optimization_Under_Uncertainty/blob/main/stochastic_pdstsp_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import heapq
from scipy.spatial.distance import cdist

# --- 1. Geração do Problema e Cenários ---

def gerar_problema(num_clientes, grid_size_km=25, pct_drone_elegivel=0.8):
    """
    Gera dados do problema com base nos parâmetros dos artigos.
    Depot no centro (0,0), clientes em [-size/2, +size/2].
    Baseado em Nguyen et al. [cite: 2136, 2137] e Mbiadou-Saleu et al. [cite: 1327]
    """
    print(f"Gerando problema: {num_clientes} clientes, grid de {grid_size_km}km, {pct_drone_elegivel*100}% elegíveis para drone.")

    # Depot na origem (centro)
    depot_coords = np.array([[0, 0]])

    # Clientes aleatórios no grid
    client_coords = np.random.uniform(
        low=-grid_size_km / 2,
        high=grid_size_km / 2,
        size=(num_clientes, 2)
    )

    # Concatena depot (índice 0) e clientes (índice 1 em diante)
    all_coords = np.vstack((depot_coords, client_coords))

    # Define elegibilidade dos drones
    drone_elegivel = np.zeros(num_clientes + 1, dtype=bool)
    num_elegiveis = int(num_clientes * pct_drone_elegivel)

    # Seleciona aleatoriamente os clientes elegíveis
    elegiveis_idx = np.random.choice(range(1, num_clientes + 1), num_elegiveis, replace=False)
    drone_elegivel[elegiveis_idx] = True

    return all_coords, drone_elegivel

def calcular_tempos_base(coords, truck_speed_kph=30, drone_speed_kph=40):
    """
    Calcula matrizes de tempo determinísticas (base).
    Caminhão: Manhattan , 30 km/h
    Drone: Euclidiana , 40 km/h
    """
    # Distâncias do Caminhão (Manhattan/cityblock)
    dist_truck = cdist(coords, coords, metric='cityblock')
    time_truck_hours = dist_truck / truck_speed_kph

    # Distâncias do Drone (Euclidiana) - Apenas de/para o depot (índice 0)
    dist_drone = cdist(coords, coords, metric='euclidean')

    # t_i^U é o tempo de ida e volta (Depot -> Cliente i -> Depot)
    time_drone_roundtrip_hours = (dist_drone[0, :] + dist_drone[:, 0]) / drone_speed_kph

    # Retorna tempos em MINUTOS
    return time_truck_hours * 60, time_drone_roundtrip_hours * 60

def obter_cenarios_estocasticos():
    """
    Define os cenários, probabilidades e multiplicadores de incerteza.
    """
    # (Nome, Probabilidade, Fator Caminhão, Fator Drone)
    cenarios = {
        "Normal": (0.5, 1.0, 1.0),
        "Tráfego Ruim": (0.3, 1.8, 1.0), # Caminhões 80% mais lentos
        "Vento Forte": (0.2, 1.0, 1.4), # Drones 40% mais lentos
    }
    return cenarios

# --- 2. Heurística PMSP (LPT) ---

def resolver_pmsp_lpt(tarefas_drone, num_drones):
    """
    Resolve o Parallel Machine Scheduling Problem (PMSP) usando LPT.
    LPT (Longest Processing Time) é uma heurística comum para PMSP[cite: 1236, 3163, 3263].

    Input:
    - tarefas_drone: Lista de tempos de duração das tarefas (já calculados para o cenário)
    - num_drones: Número de drones (máquinas)

    Output:
    - Tempo de conclusão do *último* drone (o makespan do PMSP).
    """
    if not tarefas_drone:
        return 0.0

    # Ordena tarefas da mais longa para a mais curta
    tarefas_drone.sort(reverse=True)

    # Mantém o tempo de conclusão atual de cada drone (min-heap)
    # Cada item na heap é (tempo_total_drone, drone_id)
    drone_completion_times = [(0.0, i) for i in range(num_drones)]
    heapq.heapify(drone_completion_times)

    for tarefa_duracao in tarefas_drone:
        # Pega o drone que está mais livre (menor tempo total)
        tempo_drone, drone_id = heapq.heappop(drone_completion_times)

        # Atribui a nova tarefa a ele
        novo_tempo = tempo_drone + tarefa_duracao

        # Coloca de volta na heap
        heapq.heappush(drone_completion_times, (novo_tempo, drone_id))

    # O makespan é o tempo do drone que terminou por último (o maior valor na heap)
    tempo_maximo_drone = max(t for t, _ in drone_completion_times)
    return tempo_maximo_drone

# --- 3. Avaliador Estocástico ---

def avaliar_solucao_estocastica(solucao_estagio1, num_drones, tempos_base, cenarios):
    """
    Função principal. Avalia uma solução de Estágio 1 em todos os cenários
    e calcula o tempo de conclusão esperado (o novo objetivo).
    """

    rota_caminhao = solucao_estagio1["caminhao"]
    clientes_drone = solucao_estagio1["drones"]

    tempo_base_caminhao, tempo_base_drone_roundtrip = tempos_base

    tempo_conclusao_esperado = 0.0

    print("\nAvaliando Solução nos Cenários:")
    print("-" * 30)

    for nome_cenario, (prob, fator_caminhao, fator_drone) in cenarios.items():

        # --- A. Calcular tempo do caminhão para este cenário ---
        tempo_caminhao_cenario = 0.0
        if rota_caminhao:
            for i in range(len(rota_caminhao) - 1):
                idx_de = rota_caminhao[i]
                idx_para = rota_caminhao[i+1]

                # Aplica o fator estocástico do cenário
                tempo_trecho = tempo_base_caminhao[idx_de, idx_para] * fator_caminhao
                tempo_caminhao_cenario += tempo_trecho

        # --- B. Calcular tempo dos drones para este cenário (resolver PMSP) ---
        tarefas_drone_cenario = []
        for cliente_idx in clientes_drone:
            # Aplica o fator estocástico do cenário
            tempo_tarefa = tempo_base_drone_roundtrip[cliente_idx] * fator_drone
            tarefas_drone_cenario.append(tempo_tarefa)

        tempo_max_drone_cenario = resolver_pmsp_lpt(tarefas_drone_cenario, num_drones)

        # --- C. Calcular tempo de conclusão (makespan) para este cenário ---
        tempo_conclusao_cenario = max(tempo_caminhao_cenario, tempo_max_drone_cenario)

        print(f"  Cenário: {nome_cenario} (Prob: {prob:.1f})")
        print(f"    - Tempo Caminhão: {tempo_caminhao_cenario:.2f} min")
        print(f"    - Tempo Drones (LPT): {tempo_max_drone_cenario:.2f} min")
        print(f"    - Tempo Conclusão Cenário: {tempo_conclusao_cenario:.2f} min")

        # --- D. Adicionar ao valor esperado ---
        tempo_conclusao_esperado += prob * tempo_conclusao_cenario

    return tempo_conclusao_esperado

# --- 4. Execução da Simulação ---

if __name__ == "__main__":
    # Configurações do Problema (baseado nos artigos)
    NUM_CLIENTES = 20
    NUM_DRONES = 3
    GRID_SIZE_KM = 25

    # Gera o problema (coordenadas, elegibilidade)
    # Usamos uma seed fixa para reprodutibilidade
    np.random.seed(42)
    coords, drone_elegivel = gerar_problema(NUM_CLIENTES, GRID_SIZE_KM)

    # Calcula os tempos de viagem base (determinísticos)
    tempos_base = calcular_tempos_base(coords)

    # Define os cenários estocásticos
    cenarios = obter_cenarios_estocasticos()

    # --- Definição de Duas Soluções de Estágio 1 (Alocações) para Comparar ---
    # (Índices de 1 a 20 são clientes, 0 é o depot)

    # Solução 1: Focada no Caminhão (poucos drones)
    # Atribui apenas clientes elegíveis e próximos ao depot para drones
    clientes_elegiveis = [i for i, elegivel in enumerate(drone_elegivel) if elegivel]
    clientes_drone_sol1 = [c for c in clientes_elegiveis if tempos_base[1][c] < 60][:NUM_DRONES] # Apenas os N mais rápidos
    clientes_caminhao_sol1 = [i for i in range(1, NUM_CLIENTES + 1) if i not in clientes_drone_sol1]

    # Rota simples do caminhão (heurística "vizinho mais próximo" simples)
    rota_caminhao_sol1 = [0] + sorted(clientes_caminhao_sol1, key=lambda c: coords[c][0]) + [0] # Ordena por coordenada X

    solucao_1 = {
        "nome": "Solução 1: Foco no Caminhão",
        "caminhao": rota_caminhao_sol1,
        "drones": clientes_drone_sol1
    }

    # Solução 2: Focada nos Drones (agressiva)
    # Atribui o máximo possível aos drones, caminhão faz o resto
    clientes_drone_sol2 = clientes_elegiveis # Todos elegíveis
    clientes_caminhao_sol2 = [i for i in range(1, NUM_CLIENTES + 1) if i not in clientes_drone_sol2]

    # Rota do caminhão (pode estar vazia se todos forem elegíveis)
    rota_caminhao_sol2 = [0]
    if clientes_caminhao_sol2:
        rota_caminhao_sol2 += sorted(clientes_caminhao_sol2, key=lambda c: coords[c][1]) # Ordena por Y
    rota_caminhao_sol2.append(0)

    solucao_2 = {
        "nome": "Solução 2: Foco nos Drones",
        "caminhao": rota_caminhao_sol2,
        "drones": clientes_drone_sol2
    }

    # --- Avaliar as duas soluções ---
    print(f"\n--- {solucao_1['nome']} ---")
    tempo_esperado_1 = avaliar_solucao_estocastica(solucao_1, NUM_DRONES, tempos_base, cenarios)

    print(f"\n--- {solucao_2['nome']} ---")
    tempo_esperado_2 = avaliar_solucao_estocastica(solucao_2, NUM_DRONES, tempos_base, cenarios)

    # --- Conclusão ---
    print("\n" + "="*30)
    print("RESULTADO FINAL DA SIMULAÇÃO")
    print("="*30)
    print(f"Tempo de Conclusão Esperado (Solução 1): {tempo_esperado_1:.2f} minutos")
    print(f"Tempo de Conclusão Esperado (Solução 2): {tempo_esperado_2:.2f} minutos")

    if tempo_esperado_1 < tempo_esperado_2:
        print("\nConclusão: A Solução 1 (Foco no Caminhão) é mais robusta e vence em média.")
    else:
        print("\nConclusão: A Solução 2 (Foco nos Drones) é mais robusta e vence em média.")

Gerando problema: 20 clientes, grid de 25km, 80.0% elegíveis para drone.

--- Solução 1: Foco no Caminhão ---

Avaliando Solução nos Cenários:
------------------------------
  Cenário: Normal (Prob: 0.5)
    - Tempo Caminhão: 423.27 min
    - Tempo Drones (LPT): 36.49 min
    - Tempo Conclusão Cenário: 423.27 min
  Cenário: Tráfego Ruim (Prob: 0.3)
    - Tempo Caminhão: 761.89 min
    - Tempo Drones (LPT): 36.49 min
    - Tempo Conclusão Cenário: 761.89 min
  Cenário: Vento Forte (Prob: 0.2)
    - Tempo Caminhão: 423.27 min
    - Tempo Drones (LPT): 51.08 min
    - Tempo Conclusão Cenário: 423.27 min

--- Solução 2: Foco nos Drones ---

Avaliando Solução nos Cenários:
------------------------------
  Cenário: Normal (Prob: 0.5)
    - Tempo Caminhão: 140.44 min
    - Tempo Drones (LPT): 165.14 min
    - Tempo Conclusão Cenário: 165.14 min
  Cenário: Tráfego Ruim (Prob: 0.3)
    - Tempo Caminhão: 252.80 min
    - Tempo Drones (LPT): 165.14 min
    - Tempo Conclusão Cenário: 252.80 min
  